In [ ]:
from script_util import root, fname2id, id2fname, tqdm_load, rcParams
from functools import reduce
import json
from collections import defaultdict
from tqdm.notebook import tqdm
from plot_metric.functions import BinaryClassification
import matplotlib.pyplot as plt
from collections import defaultdict
from sklearn.metrics import roc_auc_score

plt.rcParams.update(rcParams)

In [ ]:
i = 2
if not "ids" in globals():
    ids = tqdm_load(root/f"danbooru2020/{i}_id.pkl")
if not "tags" in globals():
    tags = tqdm_load(root/f"danbooru2020/{i}_tag.pkl")

In [ ]:
targets = list(map(fname2id, (root/"danbooru2020/v4").glob("**/*.png")))

In [ ]:
def load_ans(name, targets=targets):
    bizarre_ans = defaultdict(list)
    for _id in tqdm(targets):
        fname = id2fname(_id, prefix=f"v4_tagger_{name}", ext="json")
        with open(fname, "r") as f:
            data = json.load(f)
        for k, v in data.items():
            bizarre_ans[k.replace(" ", "_")].append(v)
    return bizarre_ans
i2v_ans = load_ans("i2v")
bizarre_ans = load_ans("bizarre")

In [ ]:
assert all(len(a) == 80000 for a in bizarre_ans.values()) and all(len(a) == 80000 for a in i2v_ans.values())

In [ ]:
pkeys = bizarre_ans.keys() & tags.keys() & i2v_ans.keys()
print(len(pkeys))
print(len(bizarre_ans.keys() & tags.keys()))
print(len(bizarre_ans.keys()))

In [ ]:
def met(ans=i2v_ans, query="school_uniform"):
    y_test = [_id in tags[query] for _id in targets]
    y_pred = ans[query]
    return roc_auc_score(y_test, y_pred)
met(bizarre_ans)

In [ ]:
# Only one class present in y_true. ROC AUC score is not defined in that case.
win, lose = 0, 0
for query in pkeys:
    if met(bizarre_ans, query=query) < met(i2v_ans, query=query):
        win += 1
    else:
        lose += 1
print(100*win/(win+lose))

In [ ]:
query = "school_uniform"
y_test = [_id in tags[query] for _id in targets]
plt.figure(figsize=(5,5))
bc = BinaryClassification(y_test, i2v_ans[query], labels=["Class 1", "Class 2"])
bc.plot_roc_curve(title="Receiver Operating Characteristic ", plot_threshold=False,  c_roc_curve='blue')
bc = BinaryClassification(y_test, bizarre_ans[query], labels=["Class 1", "Class 2"])
bc.plot_roc_curve(title="Receiver Operating Characteristic ", plot_threshold=False,  c_roc_curve='green')
plt.show()

In [ ]:
y_all = reduce(list.__add__, [[_id in tags[query] for _id in targets] for query in pkeys])
i2v_all = reduce(list.__add__, [i2v_ans[query] for query in pkeys])
bizarre_all = reduce(list.__add__, [bizarre_ans[query] for query in pkeys])

In [ ]:
plt.figure(figsize=(5,5))
bc = BinaryClassification(y_all, i2v_all, labels=["Class 1", "Class 2"])
bc.plot_roc_curve(title="Receiver Operating Characteristic ", plot_threshold=False,  c_roc_curve='blue')
bc = BinaryClassification(y_all, bizarre_all, labels=["Class 1", "Class 2"])
bc.plot_roc_curve(title="Receiver Operating Characteristic ", plot_threshold=False,  c_roc_curve='green')
plt.show()